In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
import tqdm
import chebyshev_fitter
import sln_letter_fit
from sln_letter_fit import FitParams, OptimizationLoggingParams
import loader, plotting

%load_ext autoreload
%autoreload 1
%aimport chebyshev_fitter, sln_letter_fit, loader, plotting

In [ ]:
# Util functions
def error(strokes, sol):
    err = np.vstack(strokes)[:, 1:] - sol['txy_from_params'][:, 1:]
    return np.sqrt(np.mean(np.sum(np.square(err), axis=1), axis=0))

# Fit trajectory

In [ ]:
poly_orders = list(range(2, 5))
fig, axes = plt.subplots(1, len(poly_orders) + 1, figsize=(20, 4))
strokes = loader.load_segments('D', index=1)

# Fit with Chebyshev
for ax, poly_order in zip(axes, poly_orders):
    sol, _, _, stroke_indices = chebyshev_fitter.fit_trajectory(strokes, p_order=poly_order)
    print('Chebyshev (order {:}) RMSE: {:}'.format(poly_order, error(strokes, sol)))
    plotting.plot_trajectory(ax, strokes, sol)
    ax.set_title('Chebyshev (order {:}) Fit, RMSE = {:.5e}'.format(poly_order, error(strokes, sol)))

# Fit with SLN
sol, _ = sln_letter_fit.fit_and_plot_trajectory(axes[-1],
                                                strokes,
                                                max_iters=50,
                                                log_history=False,
                                                pbar_description='Fitting SLN')
print('SLN RMSE: {:}'.format(error(strokes, sol)))
axes[-1].set_title('SLN Fit, RMSE = {:.5e}'.format(error(strokes, sol)));